First things first.  Load the package.

In [2]:
Sys.setenv(R_CONFIG_ACTIVE = "jupyter")
suppressMessages({
  devtools::load_all()
  data(pcd)
})

logger::log_threshold(logger::INFO, "global",index = 2)

## publishing context

This article addresses publishing context.  The workflow scenario is authoring documentation in a Jupyter notebook.  The requirement is to support rmarkdown html, rmarkdown pdf, and pkgdown article output.  Details of these document pipelines are in the appendix.

### jupyter header yaml

It can be convenient to have a copy and paste for new jupyter documents:

```yaml
---
title: "authoring"

all_doc: &shared
  pandoc_args: 
    - !with_env "--resource-path={R_HERE_HERE}/vignettes"

output:
  html_document:
    <<: *shared
    theme: cerulean
    highlight: espresso
  pdf_document:
    <<: *shared
---
```

## with_pander

`pander` converts to markdown.  `with_pander` adds logging and context dependent output.

In [3]:
# convert to markdown, with a message
pander({
  message("hello, stderr")
  pcd
})

hello, stderr





| x  |  y  |
|:--:|:---:|
| 1  |  1  |
| 2  |  4  |
| 3  |  9  |
| 4  | 16  |
| 5  | 25  |
| 6  | 36  |
| 7  | 49  |
| 8  | 64  |
| 9  | 81  |
| 10 | 100 |



In [4]:
# publishing contextualized upgrade!
with_pander({
  message("hello, stderr.  hello, log.")
  pcd
})



| x  |  y  |
|:--:|:---:|
| 1  |  1  |
| 2  |  4  |
| 3  |  9  |
| 4  | 16  |
| 5  | 25  |
| 6  | 36  |
| 7  | 49  |
| 8  | 64  |
| 9  | 81  |
| 10 | 100 |


hello, stderr.  hello, log.


## sure_plot

`with_pander` can be used with plots.  It does this by generates an image file and injecting a markdown link.  By default the image format is PNG, which is fine for HTML, but isn't ideal for PDF output.

In [5]:
# generate a PNG file
with_pander({
  plot(pcd$x, pcd$y)
})

![](plots/63f2951322858.png)

A PDF screen grab illustrates the image scaling artifacts.

![](figure/pdf-raster.png)

`sure_plot` addresses this concern via publishing context.

In [6]:
# Produce a context aware plot
sure_plot(
  {
    plot(pcd$x, pcd$y)
  },
  height = 5,
  width = 5,
  res = 96,
  par_opts = list(bg = "azure")
)

<img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAeAAAAHgCAIAAADytinCAAAACXBIWXMAAA7DAAAOwwHHb6hkAAAgAElEQVR4nO3dZ1hTZ+PH8RNBlqIkoCw3MtwDtyhuVECtgFq3UqviqAP3qntVadVasT6OohWtA0XciqNaN9bVigMVBVwQcAEC+b/I/+GhVKptIeeWfD+XL5L7Die/2F4/D3fOUKg1GgkAIJ4icgcAALwbBQ0AgqKgAUBQFDQACIqCBgBBUdAAICgKGgAERUEDgKAoaAAQFAUNAIKioAFAUBQ0AAiKggYAQVHQACAoChoABEVBA4CgKGgAEBQFDQCCoqABQFAUNAAIioIGAEFR0AAgKAoaAARFQQOAoChoABAUBQ0AgqKgAUBQFDQACIqCBgBBUdAAICgKGgAERUEDgKAoaAAQFAUNAIKioAFAUBQ0AAiKggYAQVHQACAoChoABEVBA4CgKGgAEBQFDQCCoqABQFAUNAAIioIGAEFR0AAgKAoaAARFQQOAoChoABAUBQ0AgqKgAUBQFDQACIqCBgBBUdAAICgKGgAERUEDgKAoaAAQFAUNAIKioAFAUBQ0AAiKggYAQVHQACAoChoABEVBA4CgKGgAEBQFDQCCoqABQFAUNAAIioIGAEFR0AAgKAoaAARFQQOAoChoABAUBQ0AgqKgAUBQhnIHyC09Pf1FSoqBgYGFUil3FgCQkyh70I9iY2dPmVKrUiVrExOHUqUqqFTWJiZ1HR1nTpp0PyZG7nQAIAOFWqORO4N0JSrKw81NZWnZ3svL0cVFqVJpNJpktfpOdPTBvXvVSUl7IiOr16r1L99lytixp44fz5fAAPSHianpD9u2lba21v1bC1HQXi1bmpqahmzfbmJqmmsqIyNjSN++z54+DTt06F++S9vGjQcMHly1Ro1/uR0AesX/009/2LatWs2aun9rIdagf7106du1a//czpIkGRoafhYQ0N3bO1/eyNHFpbara75sCoCeeGc16YYQa9CVnZxOHD2a1+yxI0ccnZ11mQcARCDEHvToiRP7+fndj4np7OvrVKWKhVKpUCiS1erb0dERYWF7d+1at2WL3BkBQNeEKOhOPj6h4eErliwZ7u+fc1yhUDRr2TI0PLxthw5yZQMAuQhR0JIkeXh6enh6qpOSEuLiEuLjJUkqbW1ta2+vVKnkjgYA8hCloLUslEqzYsVKWVtzogqAApWenp747JmNnZ3cQf6KEF8SSpyoAkBX7t6+3cPbu7xS2aJevQoq1Vdz56anp8sd6t2E2IP+ixNVdm7dui44OF9OVAGAhLi4ju7uoyZMCNmxo2jRog8fPBgzdOjIQYNWbdggd7R30K8TVeYFBdVv1OhfbgfAx2v6+PGSJM1atCh7JDU1tU7lyrsOH3ZycXnnjzStVWt1SAgnqvzbE1Xu3r79w5o1ec3eunnzcXz8P08J4ON36fz58dOm5RwxMTFp3rLlpfPn8ypoGQlR0NoTVTr5+Lxz9sNPVDExMfmLrxZfv34dHxf3DyMCKBQMDAwyMzNzDWZkZBgYGMiS568JUdD5daKKXZkyoyZMyGv2qzlzjI2N8y81gI9Po6ZNw3fsaNm2bfZISnLyycjIGfPny5gqL0IUNCeqANCNIV980aJevS8nTgwYPdrSyuryxYsTv/jCt2fPchUqyB3tHYQoaIkTVQDohFKlOnzmzNxp05rUqPEiJaWig8PwsWN79u8vd653E6WgtSyUSgul0qVaNbmDACi0SpUu/XVw8NfBwRkZGYaGYnVgLqKcqAIAOiZ4O0sUNAAIS4h/QPbt3n365Mm/fs3sxYt1EwYABCFEQZuamZ0+ceLiuXMmJiZ2Zcq88zUUNAB9I0RBt2jTplnLll4tW2o0mv3v25UGAD0hyhq0gYFB1+7d5U4BAAIRYg9ay7Nz5wqVKsmdAgBEIVBB25Upk9cCNADoIVGWOAAAuVDQACAoChoABEVBA4CgKGgAEBQFDQCCoqABQFAUNAAIioIGAEFR0AAgKAoaAARFQQOAoChoABAUBQ0AgqKgAUBQFDQACIqCBgBBUdAAICgKGgAERUEDgKAoaAAQFAUNAIKioAFAUBQ0AAiKggYAQVHQACAoChoABEVBA4CgKGgAEBQFDQCCoqABQFAUNAAIioIGAEFR0AAgKAoaAARlKHeA3NLT01+kpBgYGFgolXJnAQA5ibIH/Sg2dvaUKbUqVbI2MXEoVaqCSmVtYlLX0XHmpEn3Y2LkTgegYGk0mgf37t24ejUtLU3uLAIRYg/6SlSUh5ubytKyvZeXo4uLUqXSaDTJavWd6OidW7euCw7eExlZvVYtuWMCKBBHDhwYP2LE27dvi5ubP4qNDRg9euzkyYaGQrSTvIT4K5g8Zoybu3vI9u0mpqa5puYFBQ3p23dqYGDYoUOyZANQoH45eXK4v//qkJBmLVtKkhT/6NGwgQOnjRs3PyhI7mjyE2KJ49dLl/r4+/+5nSVJMjQ0/CwgIOrCBd2nAqADQQsWfLlggbadJUmytbdfGxq6ad06dVKSvMFEIERBV3ZyOnH0aF6zx44ccXR21mUeADpz7cqVZi1a5ByxUCqr1qjx+/XrMiUSiBBLHKMnTuzn53c/Jqazr69TlSoWSqVCoUhWq29HR0eEhe3dtWvdli1yZwRQIIyNjV+9epVr8PWrV8YmJrLkEYoQBd3Jxyc0PHzFkiXD/f1zjisUimYtW4aGh7ft0EGubAAKVPNWrbb9+OOkmTOzR25cvZoQF8dxAZIgBS1Jkoenp4enpzopKSEuLiE+XpKk0tbWtvb2SpXqwzdy9vTpCSNH5jX7+s0btVqdD1kB5J+JM2a0btTozZs3vQcONDc3P3706MxJk+YFBRUtWlTuaPITpaC1LJRKC6XSpVo1SZKysrLu3b1b3Nz8w/871XZ1/To4OK/ZDs2bl7SwyJ+gAPKJrb39yaiohTNn9urS5eXLlzVr1w7Zvr1ew4Zy5xKCKAV9ICLix/Xr01JTO/n49Ozff/3q1bMmT058/tzExCRw6tSxkycrFIr3bsTY2Li2q2teswZFirx/EwB0ztLKatHy5XKnEJEQBR3200/9u3WrW7++ValSX3z++aXz5zf/8MOoCRPq1Kt39tSphTNn2tja9h44UO6YAKBTQhT00vnz/YcOXbJypSRJa1etGjN06Iz580dPnChJUpv27TMzM1evWEFBA9A3QhwHfevmTQ8vL+3jzr6+kiQ1btYse7ZJ8+a3o6PlSQYA8hGioO3s7X+7dk37OObOHUmSYm7fzp69HR1dpmxZeZIBgHyEWOLoPXDg/BkzEuLjLZTK9atX161ff+bkyTZ2drXq1j176tSiWbMGDhkid0YA0DUhCnpEYODrV69C1q7NePu2k4/PwmXLxgYEfNKunXbWw9Nz7OTJ8iYEAN1TqDUauTO8W/Tvv186f97R2dm1QYN82WAZc/N5QUF9P/ssX7YGQE80rVVrdUhItZo1df/WQuxBv5OTi4uTi4vcKQBANkJ8SQgA+DMKGgAERUEDgKAoaAAQFAUNAIKioAFAUBQ0AAiKggYAQVHQACAoChoABEVBA4CgKGgAEBQFDQCCoqABQFAUNAAIioIGAEFR0AAgKAoaAARFQQOAoChoABAUBQ0AgqKgAUBQFDQACIqCBgBBUdAAICgKGgAERUEDgKAoaAAQFAUNAIKioAFAUBQ0AAiKggaQW/Tvv/fu2rVy6dIVLS27eXpeiYqSO5GeoqAB/MGFs2c93d1btGnzy7VrF6OjO/v6+nXsGHnokNy59JGh3AEAiGX6+PELvvnGp0cP7dNeAwbYlSkzafToM9euyRtMD7EHDeB/MjMzL5w54921a87BFm3aPI6Pf/7smVyp9BYFDeB/MjMzFQqFoeEffrdWKBRGRkZv376VK5XeoqAB/I+RkVElR8dTx4/nHLxx9aqBoaG1jY1cqfSWcAWdnp7+/Nk

And the corresponding screen grab from the PDF.

![](figure/pdf-vector.png)

## appendix

### publishing formats

This snippet builds all three output formats:

```r
article_name <- "authoring"
ipynb_in <- here::here("vignettes", glue("{article_name}.ipynb"))

rmd_out <- fs::path_ext_set(ipynb_in, ".Rmd")
rmd_out <- pub_ipynb_to_rmd(ipynb_in, rmd_out)

# build html and pdf as per the jupyter YAML header
knitr_set_config()
rmarkdown::render(
  input = rmd_out,
  output_format = "all"
)

# build the pkgdown article
pkgdown::build_article("authoring")
```


### incremental results

We can also build just an Rmarkdown file.  This can be processed with knitr to obtain a regular markdown file, the latter being an intermediary step in the above.


```r
article_base <- here::here("vignettes/authoring")
ipynb <- fs::path_ext_set(article_base, ".ipynb")
rmd <- fs::path_ext_set(article_base, ".Rmd")
md <- fs::path_ext_set(article_base, ".md")

pub_ipynb_to_rmd(ipynb, rmd) %>%
  knitr::knit(md)
```


However, we advice against this because, at the markdown stage, the image format is established but the final output format is not.  

We offer the following (`knitr_set_config`) to adjust the output format of `sure_plot` to meet downstream needs:

```r
# set the knitr configuration to enable 'jpeg' output from sure_plot
knitr_set_config(dev = "jpeg")
knitr::knit(rmd, md)
```